In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
import tensorflow as tf
from transformers import TFBertModel
from transformers import BertTokenizer

In [3]:
tweets = pd.read_csv("/content/drive/My Drive/processed_train.csv", usecols=['id','text', 'target'])
test = pd.read_csv("/content/drive/My Drive/processed_test.csv", usecols=['id','text'])
train_complete = pd.read_csv('/content/drive/My Drive/train_complete.csv')
test_complete = pd.read_csv('/content/drive/My Drive/test_complete.csv')
result = pd.read_csv('/content/drive/My Drive/perfect_submission.csv')

In [4]:
from sklearn.preprocessing import StandardScaler
scaled_train = StandardScaler().fit_transform(train_complete.loc[:, 'id':])
scaled_test = StandardScaler().fit_transform(test_complete.loc[:, 'id':])

In [5]:
scaled_train.shape

(7434, 218)

### Tensorflow

#### Bert

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = TFBertModel.from_pretrained('bert-base-uncased')

In [9]:
def bert_encode(data,maximum_len) :
  input_ids = []
  attention_masks = []
  for i in range(len(data.text)):
    encoded = tokenizer.encode_plus(data.text[i],
                                      add_special_tokens=True,
                                      max_length=maximum_len,
                                      pad_to_max_length=True,
                                      return_attention_mask=True, truncation=True)
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])   
  return np.array(input_ids),np.array(attention_masks)

In [10]:
train_input_ids,train_attention_masks = bert_encode(tweets, 40)
test_input_ids,test_attention_masks = bert_encode(test, 40)

print('Train length:', len(train_input_ids))
print('Test length:', len(test_input_ids))

Train length: 7434
Test length: 3263


In [11]:
def build_model(model_layer, learning_rate):
  input_ids = tf.keras.Input(shape=(40,),dtype='int32')
  attention_masks = tf.keras.Input(shape=(40,),dtype='int32')
  meta_input = tf.keras.Input(shape = (scaled_train.shape[1], ))
  
  transformer_layer = model_layer([input_ids,attention_masks])
  output = transformer_layer[1]
  output = tf.keras.layers.Concatenate()([output, meta_input])
  output = tf.keras.layers.Dense(110,activation='relu')(output)
  output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
  model = tf.keras.models.Model(inputs = [input_ids,attention_masks, meta_input],outputs = output)

  model.compile(tf.keras.optimizers.Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
  
  return model

In [12]:
model = build_model(bert, learning_rate = 6e-6)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 40)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 40, 768), (N 109482240   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 218)]        0                                 

In [13]:
history = model.fit([train_input_ids,train_attention_masks, scaled_train], tweets.target, epochs = 2, batch_size = 19, verbose=1)

Epoch 1/2
392/392 [==============================] - 3102s 8s/step - loss: 0.5048 - accuracy: 0.7542
Epoch 2/2
392/392 [==============================] - 3067s 8s/step - loss: 0.3674 - accuracy: 0.8500


In [14]:
preds_base = model.predict([test_input_ids, test_attention_masks, scaled_test])

In [15]:
prediction = []
for i in preds_base:
  if i >= 0.5:
    prediction.append(1)
  else:
    prediction.append(0)

In [ ]:
test['target'] = prediction
submit = test.loc[:, ['id', 'target']]

In [17]:
submit.to_csv('/content/drive/My Drive/submit_bert_15.csv', index=False)